# Notebook 3: Simple Search

In this notebook you learn how to compare text using semantic search.

### Optional: Install the client
You can skip this step, if you have already installed the `aleph_alpha_client`. Make sure you have the [latest pip version](https://pip.pypa.io/en/stable/installation/) installed before proceeding. 

In [ ]:
!pip install aleph_alpha_client

### Instantiate the model
Instantiate a model by providing the `model_name` and `token` for authentification. If you don't have one already, create one in your [Aleph Alpha profile](https://app.aleph-alpha.com/profile). To use semantic embeddings ([luminous-explore](https://www.aleph-alpha.com/luminous-explore-a-model-for-world-class-semantic-representation)), we need to supply `luminous-base` as the model name.

In [ ]:
from aleph_alpha_client import AlephAlphaModel
model = AlephAlphaModel.from_model_name(model_name="luminous-base", token="API_TOKEN")

### Compare the similarity of two texts

To compare two texts, you need to embed both and calculate the [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) between them. For demonstration purposes we prepared helper functions.

In [2]:
from aleph_alpha_client import SemanticEmbeddingRequest, SemanticRepresentation, Prompt
from typing import Sequence
import math

# helper function for symmetric embedding
def embed(text: str, representation: SemanticRepresentation):
    # Create a symmetric SemanticEmbeddingRequest
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=representation)
    semantic_embedding = model.semantic_embed(request)
    return semantic_embedding.embedding

# helper function to calculate similarity
def cosine_similarity(v1: Sequence[float], v2: Sequence[float]) -> float:
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

### Using semantic similarity
Now we can use the semantic similarity to find similar texts.

Let's compare two texts, one in English and one in Italian to see how it works.

In this case, both sentences have the same meaning. So they should return a high similarity score.

In [ ]:
# define the texts
text_a = "The sun is shining"
text_b = "Il sole splende"

# show the similarity
print(cosine_similarity(embed(text_a, SemanticRepresentation.Symmetric), embed(text_b, SemanticRepresentation.Symmetric)))

### The embedding
Let's also take a look at the embedding itself and how it looks.

In the cell below we print the first 100 elements of the embedding.

The embedding is 5120 elements long, so printing all of it would be quite a lot.

In [ ]:
%%time
print(embed(text_a, SemanticRepresentation.Symmetric)[:100] + "\n")
print(embed(text_b, SemanticRepresentation.Symmetric)[:100])

### Multimodal Similarity
Amazingly, as luminous supports multimodal input, we can even semantically compare texts to images with Luminous Explore.

This is not an explicitly developed feature but rather an emergent property of the model.

*Please keep in mind, that multi-modal semantic similarity is probably less robust than text to text similarity.*

In [ ]:
from aleph_alpha_client import ImagePrompt
from IPython.display import Image

url = "https://cdn-images-1.medium.com/max/1200/1*HunNdlTmoPj8EKpl-jqvBA.png"
prompt = ImagePrompt.from_url(url)
positive_text = "A neural network Architecture with Attention and Embeddings"
negative_text = "An image of a beatuiful beach"
positive_embedding = cosine_similarity(embed(positive_text, SemanticRepresentation.Symmetric), embed(prompt, SemanticRepresentation.Symmetric))
negative_embedding = cosine_similarity(embed(negative_text, SemanticRepresentation.Symmetric), embed(prompt, SemanticRepresentation.Symmetric))

# print the Image and calculated embeddings
Image(url=url, width=300, height=300)
print(f"The score for the positive example is: {positive_text_embedding}")
print(f"The score for the neagtive example is: {negative_text_embedding}")